In [44]:
!pip install -q anthropic
!pip install -q openai

In [45]:
import anthropic
import openai
import json
import pandas as pd
from google.colab import userdata
from google import genai

In [46]:
api_key = userdata.get('ANTHROPIC_API_KEY')
client = anthropic.Anthropic(api_key=api_key)

openai_api_key = userdata.get('OPENAI_API_KEY')
openai_client = openai.OpenAI(api_key = openai_api_key)

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
google_client = genai.Client(api_key=GOOGLE_API_KEY)

In [60]:
def claude_prompt(question, follow_up):
    prompt = f"""You are an expert in natural language processing. Please label the following question-follow-up pair as "1" if the follow-up asks the same or a similar question to the initial question, or if it explicitly or implicitly states that the question was not answered. Otherwise, label "0". It is okay to label 0 if the follow-up is asking for futher clarification or specifics.

Initial question: "{question}"
Follow-up: "{follow_up}"

Provide your label (1 or 0):"""
    return prompt

def claude_label(question, follow_up):
    message = client.messages.create(
        model="claude-3-7-sonnet-20250219",
        max_tokens=150,
        temperature=0,
        system="You analyze question-follow-up pairs and determine if the follow-up is asking the same/similar question or indicating the initial question wasn't answered (label '1'), or if it's a new topic or asking for further details (label '0'). Respond with ONLY the label (1 or 0).",
        messages=[
            {"role": "user", "content": claude_prompt(question, follow_up)}
        ]
    )

    response = message.content[0].text
    return response


def openai_label(question, follow_up):
    prompt = f"Label the following question-follow-up pair as '1' if the follow-up repeats the question or suggests the question was not answered. Otherwise, label '0'.\n\n. It is okay to label 0 if the follow-up is asking for futher clarification or specifics, or if the follow-up is unrelated."
    prompt += f"Question: {question}\nFollow-up: {follow_up}\nLabel:"

    response = openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are an NLP model trained to label question-follow-up pairs."},
            {"role": "user", "content": prompt}
        ]
    )

    label = response.choices[0].message.content.strip()
    return label


def google_label(question, follow_up):
    prompt = f"Label the following question-follow-up pair as '1' if the follow-up repeats the question or suggests the question was not answered. Otherwise, label '0'.\n\n. It is okay to label 0 if the follow-up is asking for futher clarification or specifics, or if the follow-up is unrelated."
    prompt += f"Question: {question}\nFollow-up: {follow_up}\nLabel:"

    response = google_client.models.generate_content(
        model="gemini-2.0-flash",
        contents=prompt,
      )

    return response.text[0]


In [49]:
with open('example_questions.json', 'r') as f:
    data = json.load(f)

df = pd.DataFrame(data)

In [50]:
df["label_claude"] = df.apply(lambda row: claude_label(row["question"], row["follow-up"]), axis=1)


In [58]:
df["label_openai"] = df.apply(lambda row: openai_label(row["question"], row["follow-up"]), axis=1)


In [61]:
df["label_google"] = df.apply(lambda row: google_label(row["question"], row["follow-up"]), axis=1)


In [62]:
df

,question,follow-up,label_claude,label_openai,label_google
0,Which crimes have gone up the most since the p...,Do you know which violent crimes have seen the...,1,1,1
1,Do you know which violent crimes have seen the...,Felony assaults have gone up a lot. Shootings ...,0,0,0
2,You have talked about supportive housing in re...,Is there a limit on how many [supportive housi...,1,1,1
3,"If you're talking with Ken Corey, are you worr...",Would you consider keeping Tisch as police com...,0,0,0
4,How would you go about convincing the parents ...,Would you say the same about Success Academy? ...,0,0,0
5,You’ve been saying that you're gonna be a good...,But how are you going to be a good manager whe...,1,1,1
6,So how do you scale it up?,And what about the property tax code? That's s...,0,0,0
7,So how are you going to stop that? If the pres...,Only after conviction? What if they’re arreste...,0,0,0
8,How do you work with the UFT [United Federatio...,Liena Zagare,0,0,0
9,Would you say the same about Success Academy? ...,Economic Development / Taxes,0,0,0
